In [1]:
from numpy import random
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set()
np.random.seed(666)
from random import sample 
from deap import tools

In [2]:
class GA():
    def __init__(self,k=2,pop_size=100, ind_size=8, n_iterations = 50):
        self.k = k
        self.pop_size = pop_size
        self.ind_size = ind_size
        self.hist = []
        self.data = np.loadtxt('file-tsp.txt')
        self.n_iterations = n_iterations
        
        self.generate_pop()
    
    def generate_pop(self):
        self.pop =  [random.permutation([i for i in range(0,self.ind_size)]) for x in range(self.pop_size) ]
        
    def tournament(self):
        new_pop = []
        for i in range(self.pop_size):
            candidates = random.choice(self.pop, 2, replace=True)

            fittest =  candidates[0] if self.evaluate(candidates[0]) < self.evaluate(candidates[1]) else candidates[1]
            new_pop.append(fittest)
        return new_pop
    
    def evaluate(self,ind):
        l = []
        for i in range(self.ind_size-1):

            dist = np.linalg.norm(self.data[ind[i]]-self.data[ind[i+1]])    
            l.append(dist)
        return l, sum(l)
    
    def avg_pop(self):
        return np.average([self.evaluate(x)[1] for x in self.pop])
    
        
    
    def order_crossover(self,p1, p2):
        #
        size = self.ind_size
        start, end = np.sort(random.choice(range(1,size - 1), 2, replace = False))
        print(f'p1: {p1}\np2: {p2}')
        print(f'start: {start}, end: {end}')
        c1 = [-1] * size
        c2 = [-1] * size
        c1[start:end] = p1[start:end]
        c2[start:end] = p2[start:end]

        p1_order = p1[end:] + p1[:end]
        p2_order = p2[end:] + p2[:end]

        missing_indices = np.append(range(end,size),range(0,start))


        pos = 0
        for index in missing_indices:
            while True:
                if(p2_order[pos] not in c1):
                    c1[index] = p2_order[pos]
                    pos += 1
                    break
                pos += 1
        pos = 0
        
        for index in missing_indices:
            while True:
                if(p1_order[pos] not in c2):
                    c2[index] = p1_order[pos]
                    pos += 1
                    break
                pos += 1    
        return c1, c2
        
        
    def run(self):
        #print(self.avg_pop())
        #for i in range(self.n_iterations):
        #self.pop = self.tournament()
        p1 = list(self.pop[0])
        p2 = list(self.pop[1])
        
        
        
        c1, c2 = self.order_crossover(p1,p2)
        
        print(f'c1: {c1}\nc2 {c2}')
        
    
        
        

In [7]:
g = GA(ind_size=50)

In [8]:
g.run()

p1: [18, 42, 14, 40, 22, 3, 43, 38, 27, 12, 30, 24, 21, 15, 34, 10, 20, 31, 32, 2, 41, 25, 5, 33, 47, 49, 6, 39, 7, 17, 29, 19, 13, 48, 8, 16, 44, 37, 4, 35, 26, 45, 23, 9, 1, 0, 36, 11, 46, 28]
p2: [28, 40, 42, 11, 8, 38, 30, 13, 26, 23, 15, 2, 5, 27, 10, 24, 45, 22, 21, 46, 17, 18, 0, 44, 37, 47, 25, 49, 1, 48, 14, 3, 31, 33, 9, 43, 6, 12, 7, 29, 19, 4, 32, 41, 34, 39, 36, 16, 20, 35]
start: 4, end: 10
c1: [30, 13, 26, 23, 22, 3, 43, 38, 27, 12, 15, 2, 5, 10, 24, 45, 21, 46, 17, 18, 0, 44, 37, 47, 25, 49, 1, 48, 14, 31, 33, 9, 6, 7, 29, 19, 4, 32, 41, 34, 39, 36, 16, 20, 35, 28, 40, 42, 11, 8]
c2 [3, 43, 27, 12, 8, 38, 30, 13, 26, 23, 24, 21, 15, 34, 10, 20, 31, 32, 2, 41, 25, 5, 33, 47, 49, 6, 39, 7, 17, 29, 19, 48, 16, 44, 37, 4, 35, 45, 9, 1, 0, 36, 11, 46, 28, 18, 42, 14, 40, 22]
